In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import pairwise_distances, pairwise_kernels
import networkx as nx
from tqdm.notebook import tqdm

import os
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning) 

import pandas as pd

import copy

In [2]:
import networkx as nx
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns

In [3]:
info = pd.read_csv(r'/Users/saravallejomengod/MathsYear4/M4R/utils/wb_info.csv', header=None)

In [4]:
goals = list(info[3].unique())
goals.remove('T')
dict_goals = {}

for goal in goals:
    g = info[4].where(info[3] == goal)

    dict_goals[goal] = [t for t in g if str(t) != 'nan']
    dict_goals[goal] = list(set(dict_goals[goal]))

## Functions to compute dHSIC

In [5]:
def width(Z):
    """
    Computes the median heuristic for the kernel bandwidth
    """
    dist_mat = pairwise_distances(Z, metric='euclidean')
    width_Z = np.median(dist_mat[dist_mat > 0])
    return width_Z

In [6]:
def make_K_list(X_list, n_samples, n_nodes):
    """
    Computes the kernel matrices of the variables in X_array, where each column represents one variable.
    Returns a list of the kernel matrices of each variable.
    """
    k_list = list(pairwise_kernels(X_list[i], metric='rbf', gamma=0.5/(width(X_list[i])**2)) for i in range(n_nodes))
    return k_list

In [8]:
def combinations(iterable, r):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield list(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield list(pool[i] for i in indices)

In [9]:
def combinations_tuple(iterable, r):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [38]:
def dHSIC(k_list):
    """
    Computes the dHSIC statistic
    """
    n_nodes = len(k_list)
    n_samples = k_list[0].shape[0]
    
    term1, term2, term3 = 1, 1, 2/n_samples    
    for j in range(n_nodes):
        term1 = term1 * k_list[j]
        term2 = term2 * np.sum(k_list[j]) / (n_samples**2)
        term3 = term3 * np.sum(k_list[j], axis=0) / n_samples
    term1_sum = np.sum(term1)
    term3_sum = np.sum(term3)
    dHSIC = term1_sum/(n_samples**2) + term2 - term3_sum
    return dHSIC

In [11]:
def dHSIC_permutation_MC(k_list, n_samples, n_nodes, stat_found, n_perms=5000, alpha=0.05):
    """
    Approximates the null distribution by permutating all variables. Using Monte Carlo approximation.
    """
    # initiating statistics
    statistics = np.zeros(n_perms)
    
    for i in range(n_perms):
        term1 = k_list[0]
        term2 = np.sum(k_list[0])/(n_samples**2)
        term3 = 2 * np.sum(k_list[0], axis=0) / (n_samples**2)

        for j in range(1, n_nodes):
            index_perm = np.random.permutation(k_list[j].shape[0])
            k_perm = k_list[j][index_perm, index_perm[:, None]]

            term1 = term1 * k_perm
            term2 = term2 * np.sum(k_perm) / (n_samples**2)
            term3 = term3 * np.sum(k_perm, axis=0) / n_samples

        term1_sum = np.sum(term1)
        term3_sum = np.sum(term3)

        statistics[i] = term1_sum/(n_samples**2) + term2 - term3_sum
    
    statistics_sort = np.sort(statistics)
    # computing 1-alpha critical value
    Bind = np.sum(stat_found==statistics_sort) + int(np.ceil((1-alpha)*(n_perms+1)))
    critical_value = statistics_sort[Bind]
    
    return critical_value

In [12]:
def joint_independence_test_MC(k_list, n_perms=5000, alpha=0.05):
    """
    Performs the independence test with HSIC and returns an accept or reject statement
    
    Inputs:
    k_list: list of Kernel matrices for each variable, each having dimensions (n_samples, n_samples)  
    n_perms: number of permutations performed when bootstrapping the null
    alpha: rejection threshold of the test
    
    Returns:
    reject: 1 if null rejected, 0 if null not rejected
    
    """
    
    n_nodes = len(k_list)
    n_samples = k_list[0].shape[0]
    
    
    # statistic and threshold
    stat = dHSIC(k_list)
    critical_value = dHSIC_permutation_MC(k_list, n_samples, n_nodes, stat) 
    
    reject = int(stat > critical_value)
    
    return stat, reject

## Pairwise

In [13]:
def HSIC_normalised_pairwise(group, groups_data, iterable):
    """
    Perform the independence test with HSIC for all possible pairwise combinations. If null hypothesis
    is rejected, compute the normalised HSIC statistic and set this as the weight of the edge between
    both variables (nodes of the graph).
    """
    #For given dictionary groups_data, take nd.array corresponding to group
    # ie. continents_prep_g_K['Europe']
    group_arr = groups_data[group]
    
    K = len(iterable)   #number of total variables (17 goals, 76 targets)
    edges = {}          #initialize dictionary with edges according to normalised HSIC
    Adj = np.eye(K)  #initialize KxK adjacency matrix 
    
    indexes = np.arange(K)    #create vector corresponding to indexes of iterable
    #find all possible 2-combinations of indexes without order
    g_combinations = list(combinations_tuple(indexes, 2))   
    
    #compute individual HSIC_XX
    HSIC_XX_sqrt = []
    for i in indexes:
        k_list_XX = list((group_arr[i], group_arr[i]))
        HSIC_XX = dHSIC(k_list_XX)
        HSIC_XX_sqrt.append(np.sqrt(HSIC_XX))
    
    for comb in g_combinations: 
        #create k_list[i] = Kernel from observed data for variable comb[i]
        k_list = list((group_arr[comb[0]], group_arr[comb[1]]))
        
        HSIC_XY = dHSIC(k_list)
        critical_value = dHSIC_permutation_MC(k_list, k_list[0].shape[0], 2, HSIC_XY)
        
        if HSIC_XY > critical_value:
            HSIC_norm = HSIC_XY/(HSIC_XX_sqrt[comb[0]]*HSIC_XX_sqrt[comb[1]])
            Adj[comb[0], comb[1]] = HSIC_norm
            Adj[comb[1], comb[0]] = HSIC_norm
            
        
    return Adj

## Create functions to compute dHcor on SDG data

In [14]:
def compute_dHSIC_XX(group, groups_data, iterable):
    """Compute dHSIC(X,X) for variable X"""
    group_arr = groups_data[group]
    K = len(iterable)
    indexes = np.arange(K)
    
    #compute individual HSIC_XX
    HSIC_XX_list = []
    for i in indexes:
        k_list_XX = list((group_arr[i], group_arr[i]))
        HSIC_XX = dHSIC(k_list_XX)
        HSIC_XX_list.append(HSIC_XX)
        
    return HSIC_XX_list


def compute_dHSIC_XXX(group, groups_data, iterable):
    """Compute dHSIC(X,X,X) for variable X"""
    group_arr = groups_data[group]
    K = len(iterable)
    indexes = np.arange(K)
    
    #compute individual HSIC_XXX
    HSIC_XXX_list = []
    for i in indexes:
        k_list_XXX = list((group_arr[i], group_arr[i], group_arr[i]))
        HSIC_XXX = dHSIC(k_list_XXX)
        HSIC_XXX_list.append(HSIC_XXX)
        
    return HSIC_XXX_list


def compute_dHSIC_XXXX(group, groups_data, iterable):
    """Compute dHSIC(X,X,X,X) for variable X"""
    group_arr = groups_data[group]
    K = len(iterable)
    indexes = np.arange(K)
    
    #compute individual HSIC_XXXX
    HSIC_XXXX_list = []
    for i in indexes:
        k_list_XXXX = list((group_arr[i], group_arr[i], group_arr[i], group_arr[i]))
        HSIC_XXXX = dHSIC(k_list_XXXX)
        HSIC_XXXX_list.append(HSIC_XXXX)
        
    return HSIC_XXXX_list

def compute_dHSIC_XXXXX(group, groups_data, iterable):
    """Compute dHSIC(X,X,X,X,X) for variable X"""
    group_arr = groups_data[group]
    K = len(iterable)
    indexes = np.arange(K)
    
    #compute individual HSIC_XXXX
    HSIC_XXXXX_list = []
    for i in indexes:
        k_list_XXXXX = list((group_arr[i], group_arr[i], group_arr[i], group_arr[i], group_arr[i]))
        HSIC_XXXXX = dHSIC(k_list_XXXXX)
        HSIC_XXXXX_list.append(HSIC_XXXXX)
        
    return HSIC_XXXXX_list

In [18]:
def dHSIC_links_MC_norm(group, groups_data, iterable, stop_after_2=False, n_perms=5000, alpha=0.05):
    #For given dictionary groups_data, take nd.array corresponding to group
    # ie. continents_prep_g_K['Europe']
    group_arr = groups_data[group]
    
    K = len(iterable)   #number of total variables (17 goals, 76 targets)
    edges = {}          #initialize dictionary with edges according to dependencies found
    Adj2 = np.zeros((K,K))  #initialize KxK adjacency matrix for d=2
    d = 2   #initial number of variables for dHSIC
    e = 0  
    
    indexes = np.arange(K)    #create vector corresponding to indexes of iterable
    #find all possible d-combinations of indexes without order
    g_combinations = list(combinations_tuple(indexes, d))   
    
    weights_3 = {}
    weights_4 = {}
    weights_5 = {}
    #iterate until no possible combinations of independent variables are left
    while len(g_combinations) >0 :
        print("combinations: ", d)
        print("number of combinations available: ", len(g_combinations))
        
        f = 0
        hsic_found = {}   #initialize dictionary with decision rule for each d-combination considered
        #iterate over all combinations considered
        for comb in g_combinations: 
            #create k_list[i] = Kernel from observed data for variable comb[i]
            k_list = []
            for i in range(d):
                k_list.append(group_arr[comb[i]])
            
            #test joint independence: if reject H0, reject=1 (dependency found)
            dHSIC_val, reject = joint_independence_test_MC(k_list, n_perms, alpha)   
            
            hsic_found[comb] = reject
            if reject == 1:
                e += 1
                f += 1
                edges[e] = tuple(iterable[i] for i in comb)  #add edge to graph according to dependency found
                
                ##### COMPUTE NORMALISATION
                ### For each size choose corresponding dictionary with the pre-computed values of dHSIC(X,...,X)
                ### for variable X.
                if d == 2:
                    HSIC_norm = dHSIC_val/np.sqrt(HSIC_XX[group][comb[0]]*HSIC_XX[group][comb[1]])
                    Adj2[comb[0], comb[1]] = HSIC_norm
                    Adj2[comb[1], comb[0]] = HSIC_norm
                    
                if d==3:
                    HSIC_norm = dHSIC_val/(HSIC_XXX[group][comb[0]]*HSIC_XXX[group][comb[1]]*HSIC_XXX[group][comb[2]])**(1/d)
                    weights_3[comb] = HSIC_norm  #add to dictionary of weights
                    
                if d==4:
                    HSIC_norm = dHSIC_val/(HSIC_XXXX[group][comb[0]]*HSIC_XXXX[group][comb[1]]*HSIC_XXXX[group][comb[2]]*HSIC_XXXX[group][comb[3]])**(1/d)
                    weights_4[comb] = HSIC_norm
                    
                if d==5:
                    HSIC_norm = dHSIC_val/(HSIC_XXXX[group][comb[0]]*HSIC_XXXX[group][comb[1]]*HSIC_XXXX[group][comb[2]]*HSIC_XXXX[group][comb[3]]*HSIC_XXXX[group][comb[4]])**(1/d)
                    weights_5[comb] = HSIC_norm
                    
                    
        print("Edges found with " ,d, "nodes: ", f)
        
        if stop_after_2==True:
            break
            
            
        d +=1 #update d
        if d==K+1:
            break #stop iteration if d is greater than available variables
        
        #Find possible d-combinations of iterable. Note that if a dependency has already been found
        #among elements of a combination <d, then we should not consider the combinations involving 
        #these elements
        g_combinations_all = list(combinations_tuple(indexes, d))
        g_combinations = copy.deepcopy(g_combinations_all)

        for comb_n in g_combinations_all:
            #consider all possible sub-combinations of d-1 elements in each comb of g_combinations_all
            gg = list(combinations_tuple(comb_n, d-1)) 
            for l in range(len(gg)):
                # for each sub_combination a dependency among its elements has already been found if 
                # that combination is not in hsic_found (so was already not considered in the previous 
                # step), or if it is but has value = 1 (there was a dependency only for the joint dist 
                # of all d-1 elements)
                if (gg[l] in hsic_found and hsic_found[gg[l]]==1) or (gg[l] not in hsic_found):
                    g_combinations.remove(comb_n)  #do not consider such combination
                    break
                   
    return edges, Adj2, weights_3, weights_4, weights_5

## Geography

In [7]:
# CHECKPOINT
continents_prep_g = pickle.load(open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/continents_prep_g.pkl', 'rb'))
continents_prep_g_K = pickle.load(open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/continents_prep_g_K.pkl', 'rb'))

In [15]:
HSIC_XX = {}
HSIC_XX['Europe'] = compute_dHSIC_XX('Europe', continents_prep_g_K, goals)
HSIC_XX['Asia'] = compute_dHSIC_XX('Asia', continents_prep_g_K, goals)
HSIC_XX['Americas'] = compute_dHSIC_XX('Americas', continents_prep_g_K, goals)
HSIC_XX['Africa'] = compute_dHSIC_XX('Africa', continents_prep_g_K, goals)

HSIC_XXX = {}
HSIC_XXX['Europe'] = compute_dHSIC_XXX('Europe', continents_prep_g_K, goals)
HSIC_XXX['Asia'] = compute_dHSIC_XXX('Asia', continents_prep_g_K, goals)
HSIC_XXX['Americas'] = compute_dHSIC_XXX('Americas', continents_prep_g_K, goals)
HSIC_XXX['Africa'] = compute_dHSIC_XXX('Africa', continents_prep_g_K, goals)

HSIC_XXXX = {}
HSIC_XXXX['Europe'] = compute_dHSIC_XXXX('Europe', continents_prep_g_K, goals)
HSIC_XXXX['Asia'] = compute_dHSIC_XXXX('Asia', continents_prep_g_K, goals)
HSIC_XXXX['Americas'] = compute_dHSIC_XXXX('Americas', continents_prep_g_K, goals)
HSIC_XXXX['Africa'] = compute_dHSIC_XXXX('Africa', continents_prep_g_K, goals)

HSIC_XXXXX = {}
HSIC_XXXXX['Europe'] = compute_dHSIC_XXXXX('Europe', continents_prep_g_K, goals)
HSIC_XXXXX['Asia'] = compute_dHSIC_XXXXX('Asia', continents_prep_g_K, goals)
HSIC_XXXXX['Americas'] = compute_dHSIC_XXXXX('Americas', continents_prep_g_K, goals)
HSIC_XXXXX['Africa'] = compute_dHSIC_XXXXX('Africa', continents_prep_g_K, goals)


In [19]:
edges_e_n, Adj2_e_n, weights_3_e, weights_4_e, weights_5_e = dHSIC_links_MC_norm('Europe', continents_prep_g_K, goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  79
combinations:  3
number of combinations available:  93
Edges found with  3 nodes:  9
combinations:  4
number of combinations available:  64
Edges found with  4 nodes:  2
combinations:  5
number of combinations available:  22
Edges found with  5 nodes:  1
combinations:  6
number of combinations available:  3
Edges found with  6 nodes:  0


In [20]:
edges_af_n, Adj2_af_n, weights_3_af, weights_4_af, weights_5_af = dHSIC_links_MC_norm('Africa', continents_prep_g_K, goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  50
combinations:  3
number of combinations available:  181
Edges found with  3 nodes:  17
combinations:  4
number of combinations available:  121
Edges found with  4 nodes:  1
combinations:  5
number of combinations available:  29
Edges found with  5 nodes:  0
combinations:  6
number of combinations available:  1
Edges found with  6 nodes:  0


In [21]:
edges_am_n, Adj2_am_n, weights_3_am, weights_4_am, weights_5_am = dHSIC_links_MC_norm('Americas', continents_prep_g_K, goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  42
combinations:  3
number of combinations available:  241
Edges found with  3 nodes:  25
combinations:  4
number of combinations available:  230
Edges found with  4 nodes:  13
combinations:  5
number of combinations available:  90
Edges found with  5 nodes:  10
combinations:  6
number of combinations available:  6
Edges found with  6 nodes:  0


In [22]:
edges_as_n, Adj2_as_n, weights_3_as, weights_4_as, weights_5_as = dHSIC_links_MC_norm('Asia', continents_prep_g_K, goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  81
combinations:  3
number of combinations available:  73
Edges found with  3 nodes:  8
combinations:  4
number of combinations available:  30
Edges found with  4 nodes:  2
combinations:  5
number of combinations available:  2
Edges found with  5 nodes:  0


Note that nodes are numbered starting from 0, so 0 corresponds to SDG 1 and 16 to SDG 17. Also note that these results slightly vary to the results found in notebooks starting with 1 and presented in the thesis, since there is a randomness factor when doing the permutation tests. However, this variation is very low (1 pairwise edge is added/missing comparing both results. But it has a bit greater impact on higher-order interactions). Normally one would compute this directly.

In [23]:
weights_3_af

{(0, 5, 16): 0.517318054591152,
 (0, 6, 13): 0.46565221826107767,
 (0, 8, 9): 0.4855786511680134,
 (1, 2, 5): 0.5117630841043357,
 (1, 4, 14): 0.32336411664661036,
 (1, 5, 6): 0.48317467340643266,
 (1, 6, 12): 0.5270250677830337,
 (1, 10, 14): 0.3032539995214665,
 (1, 14, 15): 0.36503028070463184,
 (2, 5, 10): 0.4295696150362931,
 (2, 10, 15): 0.5519834911339592,
 (4, 10, 14): 0.27449756803056524,
 (5, 10, 15): 0.4347460656101391,
 (5, 10, 16): 0.44219811564085276,
 (6, 7, 13): 0.4495866060970891,
 (10, 14, 15): 0.30585180679362606,
 (10, 14, 16): 0.3043820799018939}

In [24]:
weights_3_e

{(0, 4, 11): 0.552351596173919,
 (0, 11, 15): 0.5627071991783804,
 (1, 12, 13): 0.3906794535682024,
 (3, 11, 15): 0.6357569402738706,
 (4, 6, 11): 0.4516841797203104,
 (4, 6, 12): 0.39051880033222053,
 (5, 12, 13): 0.3523590851688421,
 (6, 11, 12): 0.3720122046173314,
 (9, 11, 12): 0.3578678840545791}

In [25]:
weights_3_am

{(0, 1, 6): 0.6047761884803207,
 (0, 1, 8): 0.6302108447198438,
 (0, 1, 9): 0.5731888868902573,
 (0, 1, 10): 0.5332564985330487,
 (0, 1, 16): 0.6724391744105209,
 (0, 4, 6): 0.5548690674375382,
 (0, 4, 9): 0.5468807220610385,
 (0, 4, 10): 0.5137497079388597,
 (0, 4, 16): 0.6450670837497104,
 (0, 8, 10): 0.5186391105539766,
 (1, 3, 6): 0.6833118984489388,
 (1, 3, 10): 0.627795301112587,
 (1, 3, 11): 0.66581928085346,
 (1, 4, 6): 0.6170896148196292,
 (1, 4, 11): 0.6054462250366052,
 (1, 4, 16): 0.7152479855906755,
 (1, 5, 8): 0.6294410556139898,
 (1, 7, 10): 0.571160941400739,
 (2, 3, 10): 0.6510798255471015,
 (2, 7, 10): 0.5904656998985888,
 (3, 8, 10): 0.6276459517509272,
 (4, 5, 8): 0.6082053198863145,
 (4, 6, 12): 0.5831154456316728,
 (4, 12, 16): 0.6818523610355206,
 (6, 13, 14): 0.32475416271481883}

In [37]:
weights_3_as

{(0, 4, 5): 0.45650625678626594,
 (1, 3, 9): 0.5245308694715953,
 (1, 3, 14): 0.40536011209370926,
 (1, 4, 5): 0.49521807996044764,
 (1, 5, 9): 0.398821708829959,
 (1, 5, 12): 0.4754667283906946,
 (2, 4, 8): 0.621002164856549,
 (2, 5, 12): 0.4860589135268572}

## Income

In [26]:
# CHECKPOINT
groups_prep_g = pickle.load(open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/groups_prep_g.pkl', 'rb'))
groups_prep_g_K = pickle.load(open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/groups_prep_g_K.pkl', 'rb'))

In [27]:
HSIC_XX['Low Income'] = compute_dHSIC_XX('Low Income', groups_prep_g_K, goals)
HSIC_XX['Lower middle Income'] = compute_dHSIC_XX('Lower middle Income', groups_prep_g_K, goals)
HSIC_XX['Upper middle Income'] = compute_dHSIC_XX('Upper middle Income', groups_prep_g_K, goals)
HSIC_XX['High Income'] = compute_dHSIC_XX('High Income', groups_prep_g_K, goals)

HSIC_XXX['Low Income'] = compute_dHSIC_XXX('Low Income', groups_prep_g_K, goals)
HSIC_XXX['Lower middle Income'] = compute_dHSIC_XXX('Lower middle Income', groups_prep_g_K, goals)
HSIC_XXX['Upper middle Income'] = compute_dHSIC_XXX('Upper middle Income', groups_prep_g_K, goals)
HSIC_XXX['High Income'] = compute_dHSIC_XXX('High Income', groups_prep_g_K, goals)

HSIC_XXXX['Low Income'] = compute_dHSIC_XXXX('Low Income', groups_prep_g_K, goals)
HSIC_XXXX['Lower middle Income'] = compute_dHSIC_XXXX('Lower middle Income', groups_prep_g_K, goals)
HSIC_XXXX['Upper middle Income'] = compute_dHSIC_XXXX('Upper middle Income', groups_prep_g_K, goals)
HSIC_XXXX['High Income'] = compute_dHSIC_XXXX('High Income', groups_prep_g_K, goals)

HSIC_XXXXX['Low Income'] = compute_dHSIC_XXXXX('Low Income', groups_prep_g_K, goals)
HSIC_XXXXX['Lower middle Income'] = compute_dHSIC_XXXXX('Lower middle Income', groups_prep_g_K, goals)
HSIC_XXXXX['Upper middle Income'] = compute_dHSIC_XXXXX('Upper middle Income', groups_prep_g_K, goals)
HSIC_XXXXX['High Income'] = compute_dHSIC_XXXXX('High Income', groups_prep_g_K, goals)

In [28]:
edges_li_n, Adj2_li_n, weights_3_li, weights_4_li, weights_5_li = dHSIC_links_MC_norm('Low Income', 
                                                                                      groups_prep_g_K, 
                                                                                      goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  29
combinations:  3
number of combinations available:  339
Edges found with  3 nodes:  14
combinations:  4
number of combinations available:  522
Edges found with  4 nodes:  5
combinations:  5
number of combinations available:  441
Edges found with  5 nodes:  0
combinations:  6
number of combinations available:  197
Edges found with  6 nodes:  0
combinations:  7
number of combinations available:  40
Edges found with  7 nodes:  0
combinations:  8
number of combinations available:  2
Edges found with  8 nodes:  0


In [29]:
edges_lmi_n, Adj2_lmi_n, weights_3_lmi, weights_4_lmi, weights_5_lmi = dHSIC_links_MC_norm('Lower middle Income', 
                                                                                           groups_prep_g_K, 
                                                                                           goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  46
combinations:  3
number of combinations available:  204
Edges found with  3 nodes:  11
combinations:  4
number of combinations available:  178
Edges found with  4 nodes:  2
combinations:  5
number of combinations available:  66
Edges found with  5 nodes:  0
combinations:  6
number of combinations available:  6
Edges found with  6 nodes:  0


In [30]:
edges_umi_n, Adj2_umi_n, weights_3_umi, weights_4_umi, weights_5_umi = dHSIC_links_MC_norm('Upper middle Income', 
                                                                                           groups_prep_g_K, 
                                                                                           goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  53
combinations:  3
number of combinations available:  171
Edges found with  3 nodes:  17
combinations:  4
number of combinations available:  118
Edges found with  4 nodes:  3
combinations:  5
number of combinations available:  29
Edges found with  5 nodes:  0


In [31]:
edges_hi_n, Adj2_hi_n, weights_3_hi, weights_4_hi, weights_5_hi = dHSIC_links_MC_norm('High Income', 
                                                                                      groups_prep_g_K, 
                                                                                      goals, stop_after_2=False)

combinations:  2
number of combinations available:  136
Edges found with  2 nodes:  104
combinations:  3
number of combinations available:  20
Edges found with  3 nodes:  3
combinations:  4
number of combinations available:  2
Edges found with  4 nodes:  0


In [32]:
weights_3_li

{(0, 4, 12): 0.6239370808692326,
 (1, 3, 6): 0.7567830838257555,
 (1, 6, 12): 0.6482964120031154,
 (1, 11, 12): 0.6667488682682163,
 (1, 11, 13): 0.6400832264922859,
 (2, 3, 5): 0.6681130629693043,
 (2, 3, 16): 0.7972400104463777,
 (2, 5, 10): 0.5914487852105873,
 (2, 10, 16): 0.7063161038246938,
 (3, 4, 12): 0.6492884307223529,
 (3, 5, 6): 0.6299411787658997,
 (3, 6, 12): 0.6678646963866877,
 (3, 10, 16): 0.7298234325828132,
 (3, 11, 12): 0.6750313470657403}

In [34]:
weights_3_lmi

{(0, 2, 13): 0.4620546229131828,
 (0, 5, 13): 0.37792117039986256,
 (0, 6, 9): 0.44296474160873256,
 (0, 8, 16): 0.6317223908936701,
 (1, 4, 9): 0.5191576951616631,
 (5, 6, 13): 0.35358162107848556,
 (5, 8, 13): 0.38288156977222665,
 (5, 13, 15): 0.41217160851517043,
 (6, 9, 15): 0.4919653145114121,
 (7, 11, 12): 0.5474540113553859,
 (8, 10, 12): 0.4645026361825696}

In [35]:
weights_3_umi

{(0, 2, 4): 0.5479393928424914,
 (0, 4, 6): 0.49091632128235474,
 (0, 4, 14): 0.2858799637348693,
 (1, 8, 13): 0.47989477588395063,
 (2, 3, 4): 0.6447820130197153,
 (2, 7, 10): 0.525601143862533,
 (3, 4, 6): 0.5780166684406807,
 (3, 4, 9): 0.49481048587931775,
 (3, 4, 14): 0.34251047036383586,
 (3, 6, 11): 0.5592412120771778,
 (3, 9, 12): 0.4985805530522867,
 (3, 11, 12): 0.5621065836689697,
 (4, 8, 9): 0.45448539290394274,
 (4, 8, 14): 0.3137443561891768,
 (4, 9, 12): 0.4240525024497302,
 (4, 9, 15): 0.4397844011643624,
 (4, 10, 14): 0.2615948782997213}

In [36]:
weights_3_hi

{(0, 12, 15): 0.32601931374922755,
 (2, 12, 14): 0.2660676425867357,
 (5, 12, 13): 0.24556083667245573}

Compute missing ones that we had found before. (This is only for my thesis, normally one would compute them all at the same time)

In [47]:
K_LI_1_4_13 = [groups_prep_g_K['Low Income'][0], groups_prep_g_K['Low Income'][3], groups_prep_g_K['Low Income'][12]]
LI_1_4_13 = dHSIC(K_LI_1_4_13)
LI_1_4_13/(HSIC_XXX['Low Income'][0]*HSIC_XXX['Low Income'][3]*HSIC_XXX['Low Income'][12])**(1/3)

0.6841105320530013

In [49]:
K_LI_2_4_6 = [groups_prep_g_K['Low Income'][1], groups_prep_g_K['Low Income'][3], groups_prep_g_K['Low Income'][5]]
LI_2_4_6 = dHSIC(K_LI_2_4_6)
LI_2_4_6/(HSIC_XXX['Low Income'][1]*HSIC_XXX['Low Income'][3]*HSIC_XXX['Low Income'][5])**(1/3)

0.680039853358151

In [50]:
K_LI_2_6_7 = [groups_prep_g_K['Low Income'][1], groups_prep_g_K['Low Income'][5], groups_prep_g_K['Low Income'][6]]
LI_2_6_7 = dHSIC(K_LI_2_6_7)
LI_2_6_7/(HSIC_XXX['Low Income'][1]*HSIC_XXX['Low Income'][5]*HSIC_XXX['Low Income'][6])**(1/3)

0.6228308988880011

In [51]:
K_LI_5_12_13 = [groups_prep_g_K['Low Income'][4], groups_prep_g_K['Low Income'][11], groups_prep_g_K['Low Income'][12]]
LI_5_12_13 = dHSIC(K_LI_5_12_13)
LI_5_12_13/(HSIC_XXX['Low Income'][4]*HSIC_XXX['Low Income'][11]*HSIC_XXX['Low Income'][12])**(1/3)

0.6021597855498495

In [53]:
K_LMI_1_9_14 = [groups_prep_g_K['Lower middle Income'][0], groups_prep_g_K['Lower middle Income'][8], 
               groups_prep_g_K['Lower middle Income'][13]]
LMI_1_9_14 = dHSIC(K_LMI_1_9_14)
LMI_1_9_14/(HSIC_XXX['Lower middle Income'][0]*HSIC_XXX['Lower middle Income'][8]*HSIC_XXX['Lower middle Income'][13])**(1/3)

0.4423548755100861

In [52]:
K_LMI_2_8_12 = [groups_prep_g_K['Lower middle Income'][1], groups_prep_g_K['Lower middle Income'][7], 
               groups_prep_g_K['Lower middle Income'][11]]
LMI_2_8_12 = dHSIC(K_LMI_2_8_12)
LMI_2_8_12/(HSIC_XXX['Lower middle Income'][1]*HSIC_XXX['Lower middle Income'][7]*HSIC_XXX['Lower middle Income'][11])**(1/3)

0.6002648988969547

In [54]:
K_LMI_6_8_12 = [groups_prep_g_K['Lower middle Income'][5], groups_prep_g_K['Lower middle Income'][7], 
               groups_prep_g_K['Lower middle Income'][11]]
LMI_6_8_12 = dHSIC(K_LMI_6_8_12)
LMI_6_8_12/(HSIC_XXX['Lower middle Income'][5]*HSIC_XXX['Lower middle Income'][7]*HSIC_XXX['Lower middle Income'][11])**(1/3)

0.4640786282402432

In [55]:
K_UMI_6_12_14 = [groups_prep_g_K['Lower middle Income'][5], groups_prep_g_K['Lower middle Income'][11], 
               groups_prep_g_K['Lower middle Income'][13]]
UMI_6_12_14 = dHSIC(K_UMI_6_12_14)
UMI_6_12_14/(HSIC_XXX['Lower middle Income'][5]*HSIC_XXX['Lower middle Income'][11]*HSIC_XXX['Lower middle Income'][13])**(1/3)

0.35956952115257407

In [56]:
K_LMI_4_11_15 = [groups_prep_g_K['Upper middle Income'][3], groups_prep_g_K['Upper middle Income'][10], 
               groups_prep_g_K['Upper middle Income'][14]]
LMI_4_11_15 = dHSIC(K_LMI_4_11_15)
LMI_4_11_15/(HSIC_XXX['Upper middle Income'][3]*HSIC_XXX['Upper middle Income'][10]*HSIC_XXX['Upper middle Income'][14])**(1/3)

0.3050612923460161

In [58]:
np.sqrt(0.23)

0.47958315233127197